In [3]:
import folium

# list of town coordinates in the format (latitude, longitude)
towns_coords = {
    'Loosdrecht': [52.2060, 5.1029],
    'Utrecht': [52.0907, 5.1214],
    'Oudewater': [52.0259, 4.8719],
    'Gouda': [52.0115, 4.7105],
    'Woerden': [52.0865, 4.9185],
    'Uithoorn': [52.2375, 4.8264],
    'Ouderkerk aan de Amstel': [52.2947, 4.9136],
    'Amsterdam': [52.3676, 4.9041],
    'Weesp': [52.3086, 5.0414],
}

# initializing map with a starting location
m = folium.Map(location=[52.1326, 5.2913], zoom_start=7)

# adding towns to the map
for town, coords in towns_coords.items():
    folium.Marker(location=coords, popup=town).add_to(m)

# save it to a file
m.save('dutch_canal_towns.html')


In [4]:
import requests
import json

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-1"="NL"][admin_level=2];
(node["waterway"="canal"](area);
 way["waterway"="canal"](area);
 rel["waterway"="canal"](area);
);
out center;
"""
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()


In [6]:
import geopandas as gpd
from shapely.geometry import Point

# convert the json to geodataframe
geometries = []
for element in data['elements']:
    if element['type'] == 'node':
        geometries.append(Point(element['lon'], element['lat']))
    elif 'center' in element:
        geometries.append(Point(element['center']['lon'], element['center']['lat']))

gdf_canals = gpd.GeoDataFrame(geometry=geometries, crs='EPSG:4326')


In [8]:
import geopy

towns = ['Loosdrecht', 'Utrecht', 'Oudewater', 'Gouda', 'Woerden', 'Uithoorn', 'Ouderkerk aan de Amstel', 'Amsterdam', 'Weesp']
geolocator = geopy.Nominatim(user_agent="your_app_name")

town_points = []
for town in towns:
    location = geolocator.geocode(town)
    town_points.append(Point(location.longitude, location.latitude))

gdf_towns = gpd.GeoDataFrame(geometry=town_points, crs='EPSG:4326')


In [9]:
import folium

m = folium.Map(location=[52.3667, 4.8945], zoom_start=7)

for idx, row in gdf_canals.iterrows():
    folium.GeoJson(row.geometry).add_to(m)

for idx, row in gdf_towns.iterrows():
    folium.Marker([row.geometry.y, row.geometry.x]).add_to(m)

m.save("map.html")
